# MDMS DEV DB

In [1]:
from sklearn import preprocessing as pp
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio as cs
init_notebook_mode(connected=True)

import cufflinks as cf
print(cf.__version__)
cf.go_offline()

cs.tools.set_credentials_file(username='arnoc', api_key='rqPrzFeAXuVfUIRKnwLC')
cs.tools.set_config_file(world_readable=True)

%matplotlib inline

0.16


<h3>INITIALIZE MDMS DB METHODS<h3/>

In [2]:
# Symbolic link from /usr/lib64/libnsl.so.2.0.0 to /usr/lib64/libnsl.so.1 required

from __future__ import print_function

#import os
#os.chdir("/usr/lib/oracle/19.3/client64/lib")

import traceback
import cx_Oracle
import numpy as np
import pandas as pd
import seaborn as sns
import time
import sys

def connectToOracleDB():
    global connection
    global cursor
    global username
    global password
    global host
    global service
    
    connection = 0
    cursor = 0
    
    host     = "dboramdms.spsi.co.za"
    username = "icemdmsudev"
    password = "V3ry$3Cr!t"
    service  = "mdms"
    
    #host     = "oracle-scan.pecgroup.local:1521" 
    #username = "arnocombrink"
    #password = "20!9o6l3MdmsU4t"
    #service  =  "iceuat"
    
    try:
        
        connection = cx_Oracle.connect(username, password, host+"/"+service)
        
        cursor = connection.cursor()
        
        print("\nOracle DB connection initialized\n")
        
    except Exception as err:
        exc_type, exc_value, exc_tb = sys.exc_info()
        traceback.print_exception(exc_type, exc_value, exc_tb)
        exit()
           
def executeSQLquery( sql ):
    global cursor
    
    print("{}".format(sql))
    
    cursor.execute( sql )
    
    return cursor.fetchall()

def getMeterReadings( meterType, meterID, fromDt, toDt ):

    #meterID = "36a878da44b6434fbe12173931dceaf1"
    #fromDt = "2019-10-01 00:00:00" 
    #toDt = "2019-10-31 23:59:59"
    
    meterTypeCol = ""
    
    if( meterType == 'ELSTER'):
        meterTypeCol = "ELS_METER_ID"
    elif( meterType == "KAMSTRUP" ):
        meterTypeCol = "KAM_METER_ID"
    elif( meterType == "ECHELON" ):
        meterTypeCol = "NES_METER_ID"
    elif( meterType == "GENERIC" ):
        meterTypeCol = "GENERIC_METER_ID"
    
    results = executeSQLquery( """SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE {} = '{}'
                                    AND ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC"""
                              .format( meterTypeCol, meterID, fromDt, toDt ) )
    
    #results = executeSQLquery( """SELECT * FROM ( SELECT METER_READING_ID, ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
    #                               WHERE NES_METER_ID = '{}'
    #                              AND ( ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND
    #                                TO_DATE('{}','YYYY-MM-DD HH24:MI:SS')) ORDER BY ENTRY_TIME ASC )
    #                                WHERE ROWNUM <= {:d}"""
    #                          .format( meterID, fromDt, toDt, days*48 ) )
    
    return results

print("MDMS Functions Initialized")

MDMS Functions Initialized


In [3]:
connectToOracleDB()


Oracle DB connection initialized



In [53]:
#### KAMSTRUP METERS
#mid = '5a17e029-cc82-40ab-80d9-b0aea5ccb05d' #30829071
#mid = '4a6baa3d-5463-409e-9789-4a323dc0b6b4' #30829119
#mid = 'a5c318f6-0541-414b-a43d-53e599168be4' #30829087
#mid = 'd4c03484-55d9-4b0e-a9e6-3ec9d7e9716d' #19675054

#### ELSTER METERS
#mid = '0cc19758-faee-400f-b221-d608ff952e56' #3514281191616
#mid = '7bc626cc-227f-4fa5-85e1-54ed21c9dec0' #3514885019338
#mid = 'e6f871d6-c1bb-4e29-b171-d23fd6ecbfb4' #3514111291545
#mid = '2943b027-74b8-4b9f-bb75-baf92a9b45d9' #3514885224631
#mid = '69f182b3-86db-43c6-bfb1-d874f746bf4b' #3514081026427
mid = 'e3b81a0d-5434-41f2-aabc-f3fa0eda30a3' #3514885012630

#### ECHELON METERS


#### GENERIC METERS

meterReadings = getMeterReadings( meterType="ELSTER", meterID=mid, fromDt='2019-11-01 00:00:00', toDt='2019-11-30 23:59:59')
meterReadings = pd.DataFrame(data=meterReadings, columns=['ENTRY_TIME','TOTAL_KWHP','TOTAL_KWHN','TOTAL_KVARP','TOTAL_KVARN'])
meterReadings.head(10)

SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE ELS_METER_ID = 'e3b81a0d-5434-41f2-aabc-f3fa0eda30a3'
                                    AND ENTRY_TIME BETWEEN TO_DATE('2019-11-01 00:00:00','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('2019-11-30 23:59:59','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC


,ENTRY_TIME,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN
0,2019-11-01 00:00:00,2.065628e+07,0.0,5289062.945,0.0
1,2019-11-01 00:30:00,2.065638e+07,0.0,5289066.792,0.0
2,2019-11-01 01:00:00,2.065649e+07,0.0,5289071.647,0.0
3,2019-11-01 01:30:00,2.065661e+07,0.0,5289078.427,0.0
4,2019-11-01 02:00:00,2.065674e+07,0.0,5289088.325,0.0
5,2019-11-01 02:30:00,2.065685e+07,0.0,5289094.791,0.0
6,2019-11-01 03:00:00,2.065696e+07,0.0,5289102.310,0.0
7,2019-11-01 03:30:00,2.065707e+07,0.0,5289107.007,0.0
8,2019-11-01 04:00:00,2.065721e+07,0.0,5289111.417,0.0
9,2019-11-01 04:30:00,2.065746e+07,0.0,5289115.208,0.0


In [84]:
fullTS = pd.date_range('2019-11-01T00:00:00Z', '2019-11-30T23:30:00Z', freq='30T').values

meterTS = meterReadings['ENTRY_TIME'].values

In [85]:
missingTS = []

for t in fullTS:
    if(t not in meterTS):
        missingTS.append(t)
    
missingTS

[numpy.datetime64('2019-11-01T06:30:00.000000000'),
 numpy.datetime64('2019-11-01T10:00:00.000000000'),
 numpy.datetime64('2019-11-01T10:30:00.000000000'),
 numpy.datetime64('2019-11-01T22:00:00.000000000'),
 numpy.datetime64('2019-11-01T22:30:00.000000000'),
 numpy.datetime64('2019-11-01T23:00:00.000000000'),
 numpy.datetime64('2019-11-02T02:00:00.000000000'),
 numpy.datetime64('2019-11-02T02:30:00.000000000'),
 numpy.datetime64('2019-11-02T03:00:00.000000000'),
 numpy.datetime64('2019-11-02T08:30:00.000000000'),
 numpy.datetime64('2019-11-02T16:30:00.000000000'),
 numpy.datetime64('2019-11-02T17:00:00.000000000'),
 numpy.datetime64('2019-11-02T17:30:00.000000000'),
 numpy.datetime64('2019-11-02T18:00:00.000000000'),
 numpy.datetime64('2019-11-03T00:30:00.000000000'),
 numpy.datetime64('2019-11-03T01:00:00.000000000'),
 numpy.datetime64('2019-11-03T02:30:00.000000000'),
 numpy.datetime64('2019-11-03T03:00:00.000000000'),
 numpy.datetime64('2019-11-03T05:00:00.000000000'),
 numpy.datet

In [63]:
x = np.arange( int( meterReadings['TOTAL_KWHP'].diff().count() ) )
y = np.array( meterReadings['TOTAL_KWHP'].diff()[1:].values )

cs = CubicSpline(x, y)

xs = np.arange(1,(30*48),0.5)

In [64]:
len(x)

1198

In [65]:
len(xs)

2878

In [66]:
fig = go.Figure(
    data=[go.Bar(y=y,x=x)],
    layout_title_text="Original Meter Consumptions"
)
fig.show()

In [67]:
fig = go.Figure(
    data=[go.Bar(y=cs(xs),x=xs)],
    layout_title_text="Cubic Spline Meter Consumptions"
)
fig.show()

1476

In [70]:
meterCons = meterReadings.diff()

mergedCons = pd.merge( meterCons, meterReadings['ENTRY_TIME'], left_index=True, right_index=True )

mergedCons.drop('ENTRY_TIME_x', axis=1, inplace=True)

mergedCons = mergedCons[1:]

mergedCons = mergedCons.rename(columns={'ENTRY_TIME_y':'TIMESTAMPS'})

mergedCons = mergedCons.set_index('TIMESTAMPS')

#mergedCons = mergedCons.loc['2019-12-01 00:00:00':'2019-12-31 23:59:59']

mergedCons['KVA'] = np.sqrt( np.square( 2*( mergedCons['TOTAL_KWHP'] - mergedCons['TOTAL_KWHN'] ) ) + np.square( 2*( mergedCons['TOTAL_KVARP'] - mergedCons['TOTAL_KVARN'] ) ) )

mergedCons = mergedCons.reset_index()

mergedCons.drop_duplicates(subset ="TIMESTAMPS", keep = 'first', inplace = True) 

mergedCons = mergedCons.set_index('TIMESTAMPS')

meterCons = mergedCons.copy()

meterCons.head()

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
TIMESTAMPS,,,,,
2019-11-01 00:30:00,45.778,0.0,12.203,0.0,94.753121
2019-11-01 01:00:00,48.204,0.0,14.391,0.0,100.612653
2019-11-01 01:30:00,47.375,0.0,12.720,0.0,98.105841
2019-11-01 02:00:00,50.921,0.0,13.590,0.0,105.406572
2019-11-01 02:30:00,58.132,0.0,13.822,0.0,119.505265


In [71]:
min_max_scaler = pp.MinMaxScaler()
kva_minmax = min_max_scaler.fit_transform(meterCons)
meterConsMinMax = pd.DataFrame(kva_minmax,columns=meterCons.columns) 
meterConsMinMax.head()

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
0,0.113838,0.0,0.119494,0.0,0.116331
1,0.141087,0.0,0.200214,0.0,0.148565
2,0.131776,0.0,0.138567,0.0,0.134775
3,0.171605,0.0,0.170663,0.0,0.174937
4,0.252600,0.0,0.179222,0.0,0.252496


In [72]:
ufResults = meterCons.copy()

ufMaxKva = ufResults['KVA'].max()

print("UNFILTERED MAX KVA: {}".format(ufMaxKva))

q3filter = 0.85

fResults = ufResults[ ufResults['KVA'] <= ufResults['KVA'].quantile(q3filter) ]

fMaxKva = fResults['KVA'].max()

print("FILTERED MAX KVA: {}" \
      .format(fMaxKva))

print("TOTAL KVA MEAN: {}" \
      .format(ufResults['KVA'].mean()))

print("TOTAL KVA STANDARD DEVIATION: {}" \
      .format(ufResults['KVA'].std()))

print("Total KVA Outlier Value (mean + 3 * standard deviation): {}" \
      .format(ufResults['KVA'].mean() + 3 * ufResults['KVA'].std()))

print("Total KVA Outlier Value (Q3 @ 85%): {}" \
      .format(ufResults['KVA'].quantile(q3filter)))


UNFILTERED MAX KVA: 255.38730609791202
FILTERED MAX KVA: 230.6831372942318
TOTAL KVA MEAN: 163.2187705248392
TOTAL KVA STANDARD DEVIATION: 59.23660671160098
Total KVA Outlier Value (mean + 3 * standard deviation): 340.9285906596421
Total KVA Outlier Value (Q3 @ 85%): 230.69682537292672


In [73]:
otResults = ufResults[ ufResults['KVA'] > ufResults['KVA'].quantile(q3filter) ]
otResults

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
TIMESTAMPS,,,,,
2019-11-01 05:30:00,116.501,0.0,31.222,0.0,241.224346
2019-11-01 06:30:00,112.014,0.0,31.532,0.0,232.735070
2019-11-01 07:00:00,117.012,0.0,32.212,0.0,242.729653
2019-11-01 07:30:00,113.840,0.0,32.103,0.0,236.559914
2019-11-01 08:30:00,110.604,0.0,33.555,0.0,231.163863
2019-11-01 10:00:00,113.162,0.0,26.187,0.0,232.304948
2019-11-04 07:00:00,118.898,0.0,25.384,0.0,243.154945
2019-11-04 08:00:00,112.156,0.0,27.018,0.0,230.728764
2019-11-04 11:00:00,113.176,0.0,28.292,0.0,233.317331


In [74]:
ufResults['KVA'].iplot(kind='bar', yTitle='Total KVA', title='Unfiltered Apparent Power Profile (KVA)')
print("UNFILTERED TOTAL KVA MAX: {:.3f}".format( ufResults['KVA'].max() ))

UNFILTERED TOTAL KVA MAX: 255.387


In [75]:
ufResults['KVA'].iplot(kind='histogram', yTitle='Total KVA', title='Unfiltered Apparent Power Histogram (KVA)')

In [76]:
fResults['KVA'].iplot(kind='bar', yTitle='Total KVA', title='Filtered Apparent Power Profile (KVA)')
print("FILTERED TOTAL KVA MAX: {:.3f}".format( fResults['KVA'].max() ))

FILTERED TOTAL KVA MAX: 230.683


In [77]:
fResults['KVA'].iplot(kind='histogram', yTitle='Total KVA', title='Filtered Apparent Power Histogram (KVA)')

In [13]:
#fResults['WEEKNAMES'] = fResults.index.weekday_name
#fResults['YEAR']      =  fResults.index.year
#fResults['MONTH']     =  fResults.index.month
#fResults['DAY_OF_YEAR']     =  fResults.index.dayofyear

#fResults.loc['2019-12'].head()

In [14]:
#plt.plot(energyCons['ENTRY_TIME'],energyCons['TOTAL_KWHP'],label="KWHP")

#sns.set(rc={'figure.figsize':(20, 7)})
#sns.barplot(x="TIMESTAMPS", y="TOTAL_KWHP", data=results)

#daily_sums = fResults['TOTAL_KWHP'].resample('D').sum()
#daily_sums.head(20)

#ax = daily_sums.plot.bar(figsize=(23,5))
#ax.set_ylabel('Total Active kWh')
#ax.set_title('Energy Consumption Load Profile')
#plt.xticks(rotation=90)